# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [7]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as sqal
from sklearn.model_selection import train_test_split

In [8]:
X_columns = 'message'
cat_col_names = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [9]:
# load data from database
engine = sqal.create_engine('sqlite:///DisasterResponse.db')
# inspector = sqal.inspect(engine)
# inspector.get_schema_names()  # -> ['main']
# inspector.get_table_names(schema='main')  # -> ['Message']
# inspector.get_columns('Message', schema='main')   # -> the goods

df = pd.read_sql_table('Message', engine)

In [10]:
# correct outlying values in `related` column
df['related'] = np.clip(df['related'], 0, 1)

In [11]:
text = df[X_columns].values
y = df[cat_col_names].values

In [12]:
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.33, random_state=42)

### 2. Write a tokenization function to process your text data

In [ ]:
import spacy
en_nlp = spacy.load('en')

In [ ]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [ ]:
# https://realpython.com/natural-language-processing-spacy-python/ was helpful here
def tokenize(text):
    doc = en_nlp(text)
    lemmas = [token.lemma_ for token in doc if token not in stopwords and not token.is_punct]
    stems = [stemmer.stem(lemma) for lemma in lemmas]
    return stems

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [ ]:
pipe = make_pipeline(TfidfVectorizer(tokenizer=tokenize, min_df=5), 
                    MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
%%time
pipe.fit(text_train, y_train)

In [ ]:
# compute rigged score
pipe.score(text_train, y_train)

In [ ]:
# compute real score
pipe.score(text_test, y_test)

Massively overfit with a test value of only 0.278, but a training value of 0.991.

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
%%time
y_pred = pipe.predict(text_test)

In [ ]:
print(classification_report(y_test, y_pred, target_names=cat_col_names))

### 6. Improve your model

####  Let's split the `Vectorizer` out of the pipeline, precompute it on the training and test sets, and write the results to disk. 
Parameter searching is a slow business and we want to speed things up as much as we can.

In [ ]:
vect = TfidfVectorizer(tokenizer=tokenize, min_df=5)

In [ ]:
%%time
# Pre-calculate X_train and X_test for testing the categorization pipeline - takes 5 minutes to calculate
X_train1 = vect.fit_transform(text_train)
X_test1 = vect.transform(text_test)

In [ ]:
X_train1.shape

In [2]:
# https://stackoverflow.com/questions/52444921/save-numpy-array-using-pickle
import pickle as pkl

In [ ]:
fileObject = open('severedPipelineXtrain.pkl', 'wb')
pkl.dump(X_train1, fileObject)
fileObject.close()

fileObject = open('severedPipelineXtest.pkl', 'wb')
pkl.dump(X_test1, fileObject)
fileObject.close()

In [3]:
fileObject = open('severedPipelineXtrain.pkl', 'rb')
X_train = pkl.load(fileObject)
fileObject.close()

fileObject = open('severedPipelineXtest.pkl', 'rb')
X_test = pkl.load(fileObject)
fileObject.close()

In [4]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [5]:
X_train.shape

(17564, 4859)

In [6]:
X_test.shape

(8652, 4859)

#### Alright.  Back to work.  Let's actually evaluate a non-Tfidf pipeline.

In [29]:
from sklearn.ensemble import AdaBoostClassifier

In [14]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
                                           RandomForestClassifier(n_jobs=-1)))
#                                           AdaBoostClassifier()))

In [15]:
%%time
pipe.fit(X_train, y_train)

CPU times: user 8.95 s, sys: 3.48 s, total: 12.4 s
Wall time: 2min 57s


Pipeline(steps=[('multioutputclassifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

In [16]:
print(pipe.score(X_test, y_test))
print(pipe.score(X_train, y_train))

0.255663430420712
0.995445228877249


At least it's not overfitting with `AdaBoostClassifier`, like it does with `RandomForestClassifier` (0.256, 0.995).

In [17]:
%%time
y_pred = pipe.predict(X_test)

CPU times: user 18.9 s, sys: 441 ms, total: 19.3 s
Wall time: 5.9 s


In [18]:
from sklearn.metrics import classification_report

In [19]:
print(classification_report(y_test, y_pred, target_names=cat_col_names, zero_division=1))

                        precision    recall  f1-score   support

               related       0.81      0.97      0.88      6598
               request       0.89      0.45      0.60      1472
                 offer       1.00      0.00      0.00        38
           aid_related       0.74      0.67      0.71      3545
          medical_help       0.67      0.08      0.14       701
      medical_products       0.78      0.09      0.17       446
     search_and_rescue       0.69      0.05      0.09       226
              security       0.00      0.00      0.00       160
              military       0.60      0.08      0.14       267
           child_alone       1.00      1.00      1.00         0
                 water       0.87      0.37      0.52       543
                  food       0.86      0.61      0.71       965
               shelter       0.81      0.42      0.55       775
              clothing       0.81      0.10      0.18       127
                 money       0.57      

#### Let's do some parameter searching...

Use grid search to find better parameters. 

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [58]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
#                                           RandomForestClassifier(n_jobs=-1)))  # This takes almost 11 minutes
                                           AdaBoostClassifier(base_estimator=
                                                              DecisionTreeClassifier(max_depth=1),
                                               n_estimators=10, learning_rate=1))) # 1 minute
#                                           LogisticRegression(C=3, max_iter=30, warm_start=True))) 
#                                                              solver='saga', penalty='elasticnet')))
                                                              # class_weight='balanced')))

In [31]:
from sklearn.model_selection import GridSearchCV

In [66]:
# param_grid = {}
# param_grid = {'multioutputclassifier__estimator__n_estimators': [10, 30, 100]} 
# param_grid = {'multioutputclassifier__estimator__learning_rate': [0.3, 1.0, 3.0]}
# param_grid = {'multioutputclassifier__estimator__C': [3.0, 10.0, 30.0]}
# param_grid = {'multioutputclassifier__estimator__l1_ratio': [0.1, 0.3, 0.7, 1.0]}
#param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_split': [2, 4, 8, 16]}
param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': [1, 2, 4, 8]}

In [67]:
grid = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1)

In [68]:
grid.estimator['multioutputclassifier'].estimator.base_estimator

DecisionTreeClassifier(max_depth=1)

In [69]:
%%time
grid.fit(X_train, y_train)

CPU times: user 16 s, sys: 173 ms, total: 16.1 s
Wall time: 2min 17s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('multioutputclassifier',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                                                                                           learning_rate=1,
                                                                                           n_estimators=10)))]),
             n_jobs=-1,
             param_grid={'multioutputclassifier__estimator__base_estimator__min_samples_leaf': [1,
                                                                                                2,
                                                                                                4,
                                                                                                8]})

25min for miniscule benefit - LR with saga not worth pursuing.  (Although the highest score fit (l1_ratio = 0.7) was only 1/3 the duration of the most expensive.  Could be because of the warm_start setting.)

For the DecisionTreeClassifier, the default value of `min_samples_split` of 2 is the best, but the effect is very weak. [2,4,8,16]

Equilvalently, for DTC, the default value of `min_samples_leaf` was best, and varying it made no difference. [1,2,4,8]

In [70]:
grid.best_params_

{'multioutputclassifier__estimator__base_estimator__min_samples_leaf': 1}

In [71]:
grid.best_score_

0.2313832708579328

In [72]:
grid.cv_results_

{'mean_fit_time': array([22.64364576, 23.66649714, 23.90333767, 23.53951702]),
 'std_fit_time': array([0.52024917, 0.06392807, 0.55559242, 0.2904497 ]),
 'mean_score_time': array([0.70242825, 0.74180546, 0.76285634, 0.71053576]),
 'std_score_time': array([0.01740296, 0.0301284 , 0.04568998, 0.09593173]),
 'param_multioutputclassifier__estimator__base_estimator__min_samples_leaf': masked_array(data=[1, 2, 4, 8],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'multioutputclassifier__estimator__base_estimator__min_samples_leaf': 1},
  {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': 2},
  {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': 4},
  {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': 8}],
 'split0_test_score': array([0.23284942, 0.23284942, 0.23284942, 0.23256476]),
 'split1_test_score': array([0.21434671, 0.21434671, 0.21434671, 0.21434671]),
 'split

In [44]:
grid.estimator.named_steps['multioutputclassifier'].estimator.base_estimator

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.